In [2]:
from torch.optim.lr_scheduler import StepLR
import pandas as pd
import numpy as np
import torch
import torch.nn as nn 
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset
import os
#import natsort
from torch.utils import data
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
import random
from tqdm.auto import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
class CustomDataSet(Dataset):
    def __init__(self, main_dir, transform, labels):
        self.main_dir = main_dir
        self.transform = transform
        all_imgs = os.listdir(main_dir)
        self.total_imgs = all_imgs#natsort.natsorted(all_imgs)
        self.labels = labels

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        image = Image.open(img_loc)#.convert("RGB")
        tensor_image = self.transform(image)
        key = os.path.basename(img_loc)[:-4]#this is how we get our ids
        #input labels as dictionary with id number as the key
        label = self.labels[key]
        return tensor_image, label

train_transform = transforms.Compose([
     transforms.Resize((48,48)),
     transforms.ToTensor(),
     transforms.Normalize(
         [0.4824],# 0.4824, 0.4824],
         [0.2142]),# 0.2142, 0.2142])
 ])


test_transform = transforms.Compose([
     transforms.Resize((48,48)),
     transforms.ToTensor(),
     transforms.Normalize(
         [0.4824],# 0.4824, 0.4824],
         [0.2142]),# 0.2142, 0.2142])
 ])

class SimpleConvnet(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(SimpleConvnet, self).__init__()

        self.input_channels = input_channels
        self.num_classes = num_classes

        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels = self.input_channels, out_channels = 64, kernel_size=5, padding=2),
            nn.ReLU()
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size = 2)
        )

        self.block3 = nn.Sequential(
            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size = 2)
        )

        self.block4 = nn.Sequential(
            nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size = 8)
        )

        #final linear layer to project into the correct number of classes
        self.fc = nn.Linear(256, 11)
        self.sig = nn.Sigmoid()
    
    def forward(self, x):
       
        x = self.block1(x)

        x = self.block2(x)

        x = self.block3(x)

        x = self.block4(x)

        x = View((-1,256))(x)
        x = self.fc(x)
        x = self.sig(x)
        output = x
        
        return output
    
    
class View(nn.Module):
    def __init__(self, shape):
        super().__init__()
        self.shape = shape

    def forward(self, x):
        return x.view(*self.shape)

## Load Test Set

In [6]:
!for entry in "../input/ranzcr-clip-catheter-line-classification/test"/*; do echo "$entry"; done > ./text.txt

In [19]:
test_names = pd.read_csv('./text.txt', header = None)

In [20]:
test_names

,0
0,../input/ranzcr-clip-catheter-line-classificat...
1,../input/ranzcr-clip-catheter-line-classificat...
2,../input/ranzcr-clip-catheter-line-classificat...
3,../input/ranzcr-clip-catheter-line-classificat...
4,../input/ranzcr-clip-catheter-line-classificat...
...,...
3577,../input/ranzcr-clip-catheter-line-classificat...
3578,../input/ranzcr-clip-catheter-line-classificat...
3579,../input/ranzcr-clip-catheter-line-classificat...
3580,../input/ranzcr-clip-catheter-line-classificat...


In [30]:
test_ids = pd.DataFrame(test_names[0].str.slice(start = 55, stop = -4)).rename(columns = {0:'ids'}).copy()
#test_ids['ids'][0]

In [31]:
#create fake labels. we do this just so we can use the CustomDataset class
test = pd.concat([test_ids, pd.DataFrame(np.zeros((len(test_ids),11)))], axis=1)
#test

In [32]:
fake_test_labels = {}
for i in range(len(test)):
    k = test.iloc[i]['ids']
    v = torch.Tensor(test.iloc[0][1:-1])
    fake_test_labels[k] = v

In [34]:
images_test = CustomDataSet('../input/ranzcr-clip-catheter-line-classification/test',test_transform, fake_test_labels)

In [35]:
batch_size = 256
test_loader = torch.utils.data.DataLoader(images_test, batch_size=batch_size,
                                                num_workers = 4)

## Load Model

In [36]:
model_file = '../input/model-state-dec2/model_state.pt'

In [37]:
model = SimpleConvnet(1,11)
model.load_state_dict(torch.load(model_file))
model = model.to(device)

In [38]:
preds_by_batch = {}
with torch.no_grad(): 
    for i, samples in enumerate(test_loader):
        # TODO: Put the inputs and targets on the write device
        data, _ = samples
        #target = target.long()
        data = data.to(device)

        # TODO: Feed forward to get the logits
        y_pred_test = model(data)
        preds_by_batch[i] = y_pred_test
        print('Predicted on batch {0}!'.format(i))

Predicted on batch 0!
Predicted on batch 1!
Predicted on batch 2!
Predicted on batch 3!
Predicted on batch 4!
Predicted on batch 5!
Predicted on batch 6!
Predicted on batch 7!
Predicted on batch 8!
Predicted on batch 9!
Predicted on batch 10!
Predicted on batch 11!
Predicted on batch 12!
Predicted on batch 13!


In [39]:
#preds_by_batch.keys()

In [40]:
#preds_by_batch[1]

In [41]:
tens = []
for i in range(len(preds_by_batch)):
    tens.append(np.array(preds_by_batch[i].cpu()))

In [42]:
preds = pd.DataFrame(np.concatenate(np.array(tens)))
preds['ids'] = test['ids'].copy()
preds = preds[['ids', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
preds = preds.rename(columns = {'ids':'StudyInstanceUID',
                        0:'ETT - Abnormal',
                        1:'ETT - Borderline',
                        2:'ETT - Normal',
                        3:'NGT - Abnormal',
                        4:'NGT - Borderline',
                        5:'NGT - Incompletely Imaged',
                        6:'NGT - Normal',
                        7:'CVC - Abnormal',
                        8:'CVC - Borderline',
                        9:'CVC - Normal',
                        10:'Swan Ganz Catheter Present'})
#preds

In [43]:
samps = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/sample_submission.csv')

In [44]:
subs = pd.merge(samps[['StudyInstanceUID']], preds, on='StudyInstanceUID', how = 'left').copy()#['StudyInstanceUID']
#subs['StudyInstanceUID'] == samps['StudyInstanceUID']

0       True
1       True
2       True
3       True
4       True
        ... 
3577    True
3578    True
3579    True
3580    True
3581    True
Name: StudyInstanceUID, Length: 3582, dtype: bool

In [45]:
subs.to_csv('./submission.csv', index = False)